In [3]:
using Pkg
if isfile("../Project.toml") && isfile("../Manifest.toml")
    Pkg.activate("..");
    ENV["PYTHON"] = "python3";
end

using Plots, JuliaProbo, Seaborn, StaticArrays
gr();

 Activating environment at `~/Programming/julia/notebook/JuliaProbo/Project.toml`


In [ ]:
mutable struct PolicyEvaluator
    pose_min::Vector{Float64}
    pose_max::Vector{Float64}
    widths::Int64
    goal::Goal
    
end